# **LIMPIEZA Y TRANSFORMACIÓN DE LOS DATOS**

Este notebook aplica las transformaciones de limpieza derivadas del EDA y realiza validaciones básicas para asegurar consistencia y calidad de los datos antes del análisis.

**Principios**
- Transformar y exportar el dataset limpio a `data/processed/`.
- Documentar cada transformación y su motivación.

In [23]:
import pandas as pd
import numpy as np
import regex as re
import matplotlib.pyplot as plt
import seaborn as sns


pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", 100)
pd.set_option("display.width", None)

#### **1. CARGA E INTEGRACIÓN DE LOS DATASETS**

- En esta sección se cargan los datasets originales y se integran en un único DataFrame de trabajo mediante la función `lectura_ficheros`.
- Esta función automatiza el proceso de integración, detectando la columna común entre ambos datasets y analizando su correspondencia para determinar el tipo de unión más adecuado.
- Este enfoque garantiza un proceso robusto y reutilizable, evitando depender de nombres de columnas específicos.
- El DataFrame resultante se utilizará como base para las siguientes fases de limpieza y validación de los datos.

In [24]:
def lectura_ficheros(path_df1, path_df2):
    """
    Carga dos datasets, detecta automáticamente la columna común entre ambos,
    analiza el grado de correspondencia de dicha clave y realiza la unión recomendada.

    Parámetros
    ----------
    path_df1 : str
        Ruta al primer fichero CSV.

    path_df2 : str
        Ruta al segundo fichero CSV.

    Devuelve
    --------
    df : pandas.DataFrame
        DataFrame resultante de la unión de ambos datasets mediante la clave común detectada.

    Qué hace
    --------
    1. Carga ambos datasets desde las rutas especificadas.
    2. Detecta automáticamente la(s) columna(s) común(es).
    3. Selecciona la primera columna común como clave de unión.
    4. Calcula el porcentaje de correspondencia de dicha clave entre ambos datasets.
    5. Determina el tipo de unión recomendado (inner, left, right u outer).
    6. Realiza el merge utilizando la clave y el tipo de unión recomendado.
    7. Muestra información básica del resultado.

    Notas
    -----
    Esta función permite automatizar el proceso de unión sin depender de un nombre de columna
    específico, haciendo el código reutilizable en distintos datasets.
    """

    print("LECTURA DE FICHEROS")

    # Carga de datos
    df1 = pd.read_csv(path_df1)
    df2 = pd.read_csv(path_df2)

    print(f"Dataset 1: {df1.shape[0]} filas × {df1.shape[1]} columnas")
    print(f"Dataset 2: {df2.shape[0]} filas × {df2.shape[1]} columnas")

    # Detectar columnas comunes
    columnas_comunes = list(set(df1.columns).intersection(df2.columns))

    if not columnas_comunes:
        raise ValueError("ERROR: No se encontraron columnas comunes entre los datasets.")

    print("\nColumnas comunes detectadas:")
    for col in columnas_comunes:
        print(f"  - {col}")

    # Seleccionar la primera columna común como clave
    key = columnas_comunes[0]
    print(f"\nClave seleccionada: {key}")

    # ==================================================
    # ANÁLISIS DE CORRESPONDENCIA
    # ==================================================

    match_df1 = df1[key].isin(df2[key]).mean() * 100
    match_df2 = df2[key].isin(df1[key]).mean() * 100

    print("\nANÁLISIS DE CORRESPONDENCIA")
    print(f"% claves de df1 presentes en df2: {match_df1:.2f}%")
    print(f"% claves de df2 presentes en df1: {match_df2:.2f}%")

    # Determinar join recomendado
    if match_df1 == 100 and match_df2 == 100:
        join_type = "inner"
        print("Correspondencia completa → recomendado: INNER JOIN")

    elif match_df1 < 100 and match_df2 == 100:
        join_type = "left"
        print("df1 tiene claves sin correspondencia → recomendado: LEFT JOIN")

    elif match_df1 == 100 and match_df2 < 100:
        join_type = "right"
        print("df2 tiene claves sin correspondencia → recomendado: RIGHT JOIN")

    else:
        join_type = "outer"
        print("Correspondencia parcial → recomendado: OUTER JOIN")

    # ==================================================
    # MERGE
    # ==================================================

    df = df1.merge(df2, on=key, how=join_type)

    print(f"\nRESULTADO DEL MERGE ({join_type.upper()})")
    print(f"Shape final: {df.shape[0]} filas × {df.shape[1]} columnas")

    return df

In [25]:
path_df1 = "../data/raw/Customer Flight Activity.csv"
path_df2 = "../data/raw/Customer Loyalty History.csv"
df = lectura_ficheros(path_df1, path_df2)
df.head()

LECTURA DE FICHEROS
Dataset 1: 405624 filas × 10 columnas
Dataset 2: 16737 filas × 16 columnas

Columnas comunes detectadas:
  - Loyalty Number

Clave seleccionada: Loyalty Number

ANÁLISIS DE CORRESPONDENCIA
% claves de df1 presentes en df2: 100.00%
% claves de df2 presentes en df1: 100.00%
Correspondencia completa → recomendado: INNER JOIN

RESULTADO DEL MERGE (INNER)
Shape final: 405624 filas × 25 columnas


,Loyalty Number,Year,Month,Flights Booked,Flights with Companions,Total Flights,Distance,Points Accumulated,Points Redeemed,Dollar Cost Points Redeemed,Country,Province,City,Postal Code,Gender,Education,Salary,Marital Status,Loyalty Card,CLV,Enrollment Type,Enrollment Year,Enrollment Month,Cancellation Year,Cancellation Month
0,100018,2017,1,3,0,3,1521,152.0,0,0,Canada,Alberta,Edmonton,T9G 1W3,Female,Bachelor,92552.0,Married,Aurora,7919.20,Standard,2016,8,NaN,NaN
1,100102,2017,1,10,4,14,2030,203.0,0,0,Canada,Ontario,Toronto,M1R 4K3,Male,College,NaN,Single,Nova,2887.74,Standard,2013,3,NaN,NaN
2,100140,2017,1,6,0,6,1200,120.0,0,0,Canada,British Columbia,Dawson Creek,U5I 4F1,Female,College,NaN,Divorced,Nova,2838.07,Standard,2016,7,NaN,NaN
3,100214,2017,1,0,0,0,0,0.0,0,0,Canada,British Columbia,Vancouver,V5R 1W3,Male,Bachelor,63253.0,Married,Star,4170.57,Standard,2015,8,NaN,NaN
4,100272,2017,1,0,0,0,0,0.0,0,0,Canada,Ontario,Toronto,P1L 8X8,Female,Bachelor,91163.0,Divorced,Star,6622.05,Standard,2014,1,NaN,NaN


#### **2. HOMOGENEIZACIÓN Y RENOMBRADO DE COLUMNAS**

- En esta sección se estandarizan los nombres de las columnas para garantizar consistencia y facilitar su uso en las fases posteriores del análisis.
- Primero, se normalizan los nombres al formato `snake_case`, eliminando espacios y caracteres especiales.
- Posteriormente, se renombran aquellas variables con nomenclatura ambigua para mejorar su claridad semántica (por ejemplo, `year` y `month` se renombran a `flight_year` y `flight_month`).
- Este proceso mejora la legibilidad del dataset y facilita su manipulación en el flujo de limpieza y análisis.

In [26]:
def normalizar_nombres_columnas(lista_columnas, mostrar_resumen=True):
    """
    Normaliza los nombres de las columnas a formato snake_case.

    El proceso incluye:
    - Eliminar espacios al inicio y al final
    - Reemplazar espacios intermedios y caracteres especiales por guiones bajos
    - Convertir de CamelCase/PascalCase a snake_case
    - Convertir todo a minúsculas
    - Eliminar guiones bajos duplicados

    Parámetros
    ----------
    lista_columnas : list
        Lista con los nombres originales de las columnas.

    mostrar_resumen : bool, default=True
        Si True, muestra el resumen de cambios realizados.

    Devuelve
    --------
    list
        Lista con los nombres normalizados en formato snake_case.

    Ejemplo
    -------
    >>> normalizar_nombres_columnas(['Loyalty Number', 'CLV'])
    ['loyalty_number', 'clv']
    """
       
    nombres_normalizados = []

    for nombre in lista_columnas:

        # 1. eliminar espacios extremos
        limpia = nombre.strip()

        # 2. reemplazar espacios y caracteres especiales por _
        limpia = re.sub(r'[^0-9a-zA-Z]+', '_', limpia)

        # 3. convertir a minúsculas
        limpia = limpia.lower()

        # 4. eliminar _ duplicados
        limpia = re.sub(r'_+', '_', limpia)

        # 5. eliminar _ inicial o final
        limpia = limpia.strip('_')

        nombres_normalizados.append(limpia)

    if mostrar_resumen:
        print("Normalización de nombres de columnas finalizada.")
        print(f"Total columnas procesadas: {len(nombres_normalizados)}")
        print("Resumen de cambios:")
        for orig, nuevo in zip(lista_columnas, nombres_normalizados):
            print(f"  '{orig}' → '{nuevo}'")

    return nombres_normalizados

In [27]:
df.columns = normalizar_nombres_columnas(df.columns.tolist())

Normalización de nombres de columnas finalizada.
Total columnas procesadas: 25
Resumen de cambios:
  'Loyalty Number' → 'loyalty_number'
  'Year' → 'year'
  'Month' → 'month'
  'Flights Booked' → 'flights_booked'
  'Flights with Companions' → 'flights_with_companions'
  'Total Flights' → 'total_flights'
  'Distance' → 'distance'
  'Points Accumulated' → 'points_accumulated'
  'Points Redeemed' → 'points_redeemed'
  'Dollar Cost Points Redeemed' → 'dollar_cost_points_redeemed'
  'Country' → 'country'
  'Province' → 'province'
  'City' → 'city'
  'Postal Code' → 'postal_code'
  'Gender' → 'gender'
  'Education' → 'education'
  'Salary' → 'salary'
  'Marital Status' → 'marital_status'
  'Loyalty Card' → 'loyalty_card'
  'CLV' → 'clv'
  'Enrollment Type' → 'enrollment_type'
  'Enrollment Year' → 'enrollment_year'
  'Enrollment Month' → 'enrollment_month'
  'Cancellation Year' → 'cancellation_year'
  'Cancellation Month' → 'cancellation_month'


In [28]:
def renombrar_columnas_semanticas(df, mostrar_resumen=True):
    """
    Renombra columnas para mejorar la claridad semántica y eliminar ambigüedades.

    Este paso se aplica después de la normalización a snake_case y permite
    asignar nombres más descriptivos a variables cuyo significado no es
    evidente o puede generar confusión.

    Parámetros
    ----------
    df : pandas.DataFrame
        DataFrame cuyas columnas serán renombradas.

    mostrar_resumen : bool, default=True
        Si True, muestra el resumen de los cambios realizados.

    Devuelve
    --------
    pandas.DataFrame
        DataFrame con las columnas renombradas.

    Cambios aplicados
    -----------------
    - year → flight_year
    - month → flight_month
    - clv → customer_lifetime_value

    Notas
    -----
    Este paso mejora la interpretabilidad del dataset y evita ambigüedades,
    especialmente cuando existen múltiples variables relacionadas con fechas.
    """
    
    # Se define un diccionario de mapeo donde la clave representa el nombre original de la columna
    # y el valor el nuevo nombre asignado, con el objetivo de mejorar la claridad semántica.
    rename_map = {
        "year": "flight_year",
        "month": "flight_month",
        "clv": "customer_lifetime_value"
    }

    # Mediante el método .rename(), se aplican estos cambios al DataFrame, alineando la nomenclatura
    # con el estándar definido previamente en las funciones de homogeneización de columnas.
    df.rename(columns=rename_map, inplace=True)

    if mostrar_resumen:
        print("Renombrado semántico de columnas aplicado:")
        for original, nuevo in rename_map.items():
            if original in df.columns or nuevo in df.columns:
                print(f"  '{original}' → '{nuevo}'")

    return df

In [29]:
df = renombrar_columnas_semanticas(df)
df.head()

Renombrado semántico de columnas aplicado:
  'year' → 'flight_year'
  'month' → 'flight_month'
  'clv' → 'customer_lifetime_value'


,loyalty_number,flight_year,flight_month,flights_booked,flights_with_companions,total_flights,distance,points_accumulated,points_redeemed,dollar_cost_points_redeemed,country,province,city,postal_code,gender,education,salary,marital_status,loyalty_card,customer_lifetime_value,enrollment_type,enrollment_year,enrollment_month,cancellation_year,cancellation_month
0,100018,2017,1,3,0,3,1521,152.0,0,0,Canada,Alberta,Edmonton,T9G 1W3,Female,Bachelor,92552.0,Married,Aurora,7919.20,Standard,2016,8,NaN,NaN
1,100102,2017,1,10,4,14,2030,203.0,0,0,Canada,Ontario,Toronto,M1R 4K3,Male,College,NaN,Single,Nova,2887.74,Standard,2013,3,NaN,NaN
2,100140,2017,1,6,0,6,1200,120.0,0,0,Canada,British Columbia,Dawson Creek,U5I 4F1,Female,College,NaN,Divorced,Nova,2838.07,Standard,2016,7,NaN,NaN
3,100214,2017,1,0,0,0,0,0.0,0,0,Canada,British Columbia,Vancouver,V5R 1W3,Male,Bachelor,63253.0,Married,Star,4170.57,Standard,2015,8,NaN,NaN
4,100272,2017,1,0,0,0,0,0.0,0,0,Canada,Ontario,Toronto,P1L 8X8,Female,Bachelor,91163.0,Divorced,Star,6622.05,Standard,2014,1,NaN,NaN


#### **3. ELIMINACIÓN DE FILAS DUPLICADAS**

- En esta sección se identifican y eliminan filas completamente duplicadas del dataset.
- Se consideran duplicados aquellos registros que presentan valores idénticos en todas las columnas.
- La eliminación de estos registros evita redundancias y garantiza la integridad del conjunto de datos.
- Este proceso no afecta a registros válidos derivados de la estructura temporal del dataset, donde un mismo cliente puede aparecer en múltiples periodos.

In [30]:
def eliminar_filas_duplicadas(df, keep='first'):
    """
    Elimina filas completamente duplicadas del DataFrame.

    Se consideran duplicados aquellas filas que tienen valores idénticos en
    todas las columnas.

    Parámetros
    ----------
    df : pandas.DataFrame
        DataFrame a procesar.

    keep : {'first', 'last', False}, default='first'
        Determina qué ocurrencia conservar:
        - 'first' : conserva la primera aparición
        - 'last'  : conserva la última aparición
        - False   : elimina todas las ocurrencias duplicadas

    Devuelve
    --------
    pandas.DataFrame
        DataFrame sin filas duplicadas.
    """

    print("ELIMINACIÓN DE FILAS DUPLICADAS")
    # Crear copia de seguridad
    df = df.copy()

    filas_originales = df.shape[0]

    #Elimina duplicados
    df.drop_duplicates(keep=keep, inplace=True)

    filas_tras_depuracion = df.shape[0]
    filas_eliminadas = filas_originales - filas_tras_depuracion

    print(f"Filas iniciales: {filas_originales}")
    print(f"Filas eliminadas: {filas_eliminadas}")
    print(f"Filas finales: {filas_tras_depuracion}")

    if filas_eliminadas == 0:
        print("No se detectaron duplicados exactos.")
    else:
        print("Duplicados eliminados correctamente.")

    return df

In [31]:
df = eliminar_filas_duplicadas(df)
df.head()

ELIMINACIÓN DE FILAS DUPLICADAS
Filas iniciales: 405624
Filas eliminadas: 1864
Filas finales: 403760
Duplicados eliminados correctamente.


,loyalty_number,flight_year,flight_month,flights_booked,flights_with_companions,total_flights,distance,points_accumulated,points_redeemed,dollar_cost_points_redeemed,country,province,city,postal_code,gender,education,salary,marital_status,loyalty_card,customer_lifetime_value,enrollment_type,enrollment_year,enrollment_month,cancellation_year,cancellation_month
0,100018,2017,1,3,0,3,1521,152.0,0,0,Canada,Alberta,Edmonton,T9G 1W3,Female,Bachelor,92552.0,Married,Aurora,7919.20,Standard,2016,8,NaN,NaN
1,100102,2017,1,10,4,14,2030,203.0,0,0,Canada,Ontario,Toronto,M1R 4K3,Male,College,NaN,Single,Nova,2887.74,Standard,2013,3,NaN,NaN
2,100140,2017,1,6,0,6,1200,120.0,0,0,Canada,British Columbia,Dawson Creek,U5I 4F1,Female,College,NaN,Divorced,Nova,2838.07,Standard,2016,7,NaN,NaN
3,100214,2017,1,0,0,0,0,0.0,0,0,Canada,British Columbia,Vancouver,V5R 1W3,Male,Bachelor,63253.0,Married,Star,4170.57,Standard,2015,8,NaN,NaN
4,100272,2017,1,0,0,0,0,0.0,0,0,Canada,Ontario,Toronto,P1L 8X8,Female,Bachelor,91163.0,Divorced,Star,6622.05,Standard,2014,1,NaN,NaN


#### **4. CORRECCIÓN DE TIPOS DE DATOS**

- En esta sección se corrigen los tipos de datos de aquellas variables cuyo tipo original no es el más adecuado según su naturaleza.
- Las variables discretas con valores nulos se convierten a tipo entero nullable (`Int64`), mientras que las variables continuas o monetarias se convierten a tipo `float`.
- Esta corrección garantiza una representación coherente de los datos y evita problemas en fases posteriores de análisis.
- El proceso se implementa mediante un diccionario de mapeo columna → tipo, facilitando su reutilización y mantenimiento.

In [32]:
def corregir_tipos_datos(df, mostrar_resumen=True):
    """
    Corrige los tipos de datos de las columnas de un DataFrame según su naturaleza.

    Esta función permite convertir variables a su tipo más adecuado utilizando
    un diccionario de mapeo columna → tipo. Es especialmente útil para:

    - Convertir variables discretas con nulos a entero nullable (Int64)
    - Convertir variables discretas sin nulos a entero (int64)
    - Convertir variables continuas o monetarias a float (float64)

    Parámetros
    ----------
    df : pandas.DataFrame
        DataFrame cuyas columnas serán convertidas.

    mostrar_resumen : bool, default=True
        Si True, muestra el resumen de los cambios realizados.

    Devuelve
    --------
    pandas.DataFrame
        DataFrame con los tipos de datos corregidos.

    Notas
    -----
    Solo se aplican conversiones a columnas existentes en el DataFrame,
    permitiendo que la función sea reutilizable en distintos datasets.
    """

    df = df.copy()

    # Diccionario columna → tipo destino
    dtype_map = {
        # Variables discretas con nulos → entero nullable
        "cancellation_year": "Int64",
        "cancellation_month": "Int64",
        # Variables discretas sin nulos → entero
        "points_accumulated": "int64",
        # Variables continuas / monetarias → float
        "distance": "float64",
        "dollar_cost_points_redeemed": "float64"
    }

    if mostrar_resumen:
        print("CORRECCIÓN DE TIPOS DE DATOS")
        print("Columnas convertidas:")

    # Aplicar conversiones solo si la columna existe
    for col, dtype in dtype_map.items():
        if col in df.columns:
            dtype_original = df[col].dtype
            df[col] = df[col].astype(dtype)
            
            if mostrar_resumen:
                print(f"  {col}: {dtype_original} → {dtype}")

    return df


In [33]:
df = corregir_tipos_datos(df)

display(df.dtypes)

CORRECCIÓN DE TIPOS DE DATOS
Columnas convertidas:
  cancellation_year: float64 → Int64
  cancellation_month: float64 → Int64
  points_accumulated: float64 → int64
  distance: int64 → float64
  dollar_cost_points_redeemed: int64 → float64


loyalty_number                   int64
flight_year                      int64
flight_month                     int64
flights_booked                   int64
flights_with_companions          int64
total_flights                    int64
distance                       float64
points_accumulated               int64
points_redeemed                  int64
dollar_cost_points_redeemed    float64
country                         object
province                        object
city                            object
postal_code                     object
gender                          object
education                       object
salary                         float64
marital_status                  object
loyalty_card                    object
customer_lifetime_value        float64
enrollment_type                 object
enrollment_year                  int64
enrollment_month                 int64
cancellation_year                Int64
cancellation_month               Int64
dtype: object

#### **5. TRATAMIENTO DE VALORES NEGATIVOS**

- En esta sección se identifican y corrigen valores negativos en variables cuya naturaleza no admite este tipo de valores.
- Los valores negativos se consideran inconsistencias derivadas de errores de calidad de datos, ya que no representan situaciones válidas desde el punto de vista analítico.
- Estos valores son convertidos a valores nulos (`NaN`) para permitir su tratamiento posterior mediante técnicas de imputación, evitando eliminar registros completos y preservando el volumen de información disponible.
- Este tratamiento se aplica específicamente sobre la variable `salary`, que representa el ingreso anual estimado del cliente y no puede tomar valores negativos.

In [34]:
def convertir_negativos_a_nulos(df, columna, mostrar_resumen=True):
    """
    Convierte valores negativos de una columna numérica en valores nulos (NaN).

    Parámetros
    ----------
    df : pandas.DataFrame
        DataFrame a procesar.

    columna : str
        Nombre de la columna donde se corregirán los valores negativos.

    mostrar_resumen : bool, default=True
        Si True, muestra el número de valores corregidos.

    Devuelve
    --------
    pandas.DataFrame
        DataFrame con los valores negativos convertidos a NaN.

    Qué hace
    --------
    - Identifica valores negativos en la columna especificada.
    - Sustituye dichos valores por NaN, al considerarse inconsistentes con la
      naturaleza de la variable.
    - Muestra un resumen del número de valores corregidos (opcional).
    """

    df = df.copy()
    
    # Calcular el número de registros con valores negativos en la columna especificada,
    # lo que permite cuantificar el alcance de la corrección aplicada
    n_negativos = (df[columna] < 0).sum()
    
    # Reemplazar los valores negativos por NaN, ya que representan valores inválidos
    # según la naturaleza de la variable y deben ser tratados como datos faltantes
    df.loc[df[columna] < 0, columna] = np.nan

    if mostrar_resumen:
        print(f"{columna}: {n_negativos} valores negativos convertidos a NaN")

    return df

In [35]:
# Conversión de valores negativos a nulos en la variable salary
df = convertir_negativos_a_nulos(df, "salary")

# Verificar resultado
display(df["salary"].describe())

salary: 480 valores negativos convertidos a NaN


count    301020.000000
mean      79441.628829
std       34704.340158
min       15609.000000
25%       59278.000000
50%       73523.000000
75%       88626.000000
max      407228.000000
Name: salary, dtype: float64

#### **6. TRATAMIENTO DE VALORES NULOS**

- En esta sección se analiza y trata la presencia de valores nulos, aplicando distintas estrategias en función de la naturaleza y significado de cada variable.
- A partir del análisis exploratorio (EDA), se identificaron dos casuísticas diferenciadas que requieren tratamientos distintos.

**Variables que requieren imputación**

- La variable `salary` presenta valores nulos que corresponden a información faltante, no a una condición real del cliente.
- Dado que el salario es una variable continua con relevancia analítica, estos valores se imputan utilizando la mediana calculada por grupos de clientes con características similares (`education` y `loyalty_card`).
- Este enfoque permite preservar la estructura socioeconómica del dataset y evitar sesgos derivados de imputaciones globales.

**Variables cuyos valores nulos representan una condición válida**

- Las variables `cancellation_year` y `cancellation_month` presentan un alto porcentaje de valores nulos.
- Sin embargo, estos valores nulos no representan errores ni información faltante, sino que indican que el cliente no ha cancelado su membresía en el programa de fidelización.
- Por este motivo, estos valores se mantienen como nulos, ya que constituyen información válida y relevante desde el punto de vista analítico.
- Se crea la variable `customer_status` para identificar si el cliente se encuentra activo o ha cancelado su membresía, facilitando el análisis del comportamiento y la segmentación entre clientes activos y cancelados.

- Este tratamiento diferenciado garantiza la coherencia semántica de los datos y asegura una preparación adecuada del dataset para fases posteriores de análisis.


In [36]:
def imputar_salary_por_grupos(df, columna="salary", mostrar_resumen=True):
    """
    Imputa valores nulos en una variable continua utilizando la mediana
    calculada por grupos de variables estructurales relacionadas.

    En este caso, la imputación se realiza de forma jerárquica:
    Mediana por grupo de education y loyalty_card

    Este enfoque permite imputar valores de forma más realista, preservando
    las diferencias salariales entre segmentos de clientes y evitando el uso
    exclusivo de la mediana global.

    Parámetros
    ----------
    df : pandas.DataFrame
        DataFrame que contiene la variable a imputar.

    columna : str, default="salary"
        Nombre de la columna sobre la que se realizará la imputación.

    mostrar_resumen : bool, default=True
        Si True, muestra un resumen del proceso de imputación.

    Retorna
    -------
    pandas.DataFrame
        DataFrame con los valores imputados.
    """
     
    df = df.copy()

    nulos_antes = df[columna].isnull().sum()

    # 1) Mediana por grupos (si el grupo tiene mediana NaN, no imputa)
    mediana_grupo = df.groupby(["education", "loyalty_card"])[columna].transform("median")
    df[columna] = df[columna].fillna(mediana_grupo)

    # 2) Fallback por education (para los grupos que no se hayan podido imputar en el paso anterior)
    mediana_edu = df.groupby("education")[columna].transform("median")
    df[columna] = df[columna].fillna(mediana_edu)

    # 3) Fallback final: mediana global (para los nulos restantes)
    mediana_global = df[columna].median()
    df[columna] = df[columna].fillna(mediana_global)

    nulos_despues = df[columna].isnull().sum()

    if mostrar_resumen:
        print("IMPUTACIÓN DE SALARY FINALIZADA")
        print(f"Valores nulos antes: {nulos_antes:,}")
        print(f"Valores imputados: {nulos_antes - nulos_despues:,}")
        print(f"Valores nulos restantes: {nulos_despues:,}")

    return df

In [37]:
df = imputar_salary_por_grupos(df)
df['salary'].describe()

IMPUTACIÓN DE SALARY FINALIZADA
Valores nulos antes: 102,740
Valores imputados: 102,740
Valores nulos restantes: 0


count    403760.000000
mean      77922.166857
std       30078.197775
min       15609.000000
25%       64001.000000
50%       73479.000000
75%       82940.000000
max      407228.000000
Name: salary, dtype: float64

In [38]:
def clasificar_estado_cliente(valor):
    """
    Clasifica el estado del cliente en función del año de cancelación.

    Devuelve "Active" si el cliente no presenta año de cancelación (NaN),
    y "Cancelled" si el cliente ha cancelado su membresía.
    """
    
    if pd.isna(valor):
        return "Active"
    else:
        return "Cancelled"


In [39]:
df["customer_status"] = df["cancellation_year"].apply(clasificar_estado_cliente)

#### **7. LIMPIEZA DE VARIABLES CATEGÓRICAS**

- En esta sección se estandariza el formato de las variables categóricas.
- Se eliminan espacios al inicio y al final de los valores y se aplica un formato homogéneo de capitalización.
- Este proceso garantiza la consistencia de las categorías y evita la duplicación de valores equivalentes con distinta representación.
- Aunque durante el EDA no se detectaron inconsistencias en el formato de las variables categóricas, se aplica este proceso como medida preventiva para garantizar la consistencia del dataset y asegurar un flujo de limpieza automatizado y robusto.

In [40]:
def limpiar_categoricas(df, columnas=None):
    """
    Limpia variables categóricas eliminando espacios y unificando el formato del texto.

    Aplica .str.strip().str.title() para garantizar consistencia en las categorías.

    Parámetros
    ----------
    df : pandas.DataFrame
        DataFrame a procesar.

    columnas : list, default=None
        Columnas a limpiar. Si es None, se aplicará a todas las columnas categóricas.

    Devuelve
    --------
    pandas.DataFrame
        DataFrame con las variables categóricas limpias.
    """

    df = df.copy()

    # Si no se especifican columnas, detectar todas las categóricas
    if columnas is None:
        columnas = df.select_dtypes(include="O").columns

    # Aplicar limpieza básica
    for col in columnas:
        df[col] = df[col].str.strip().str.title()

    return df

In [41]:
df = limpiar_categoricas(df)

#### **8. REVISIÓN DE INCOHERENCIAS LÓGICAS**

- En esta sección se validan las relaciones lógicas entre variables relacionadas con la actividad de vuelos.
- Se comprueba la coherencia entre el número de vuelos, la distancia recorrida y los puntos acumulados y redimidos.
- Esta validación permite detectar posibles errores derivados de inconsistencias en el registro de la información.
- No se detectaron incoherencias significativas, lo que confirma la consistencia estructural del dataset.

In [42]:
def revisar_incoherencias_vuelos(df):
    """
    Detecta posibles incoherencias lógicas en las variables relacionadas.
    """

    df = df.copy()

    incoherencias = {}

    incoherencias["booked > total"] = (df["flights_booked"] > df["total_flights"]).sum()

    incoherencias["companions > total"] = (
        df["flights_with_companions"] > df["total_flights"]
    ).sum()

    incoherencias["distance > 0 pero total_flights = 0"] = (
        (df["total_flights"] == 0) & (df["distance"] > 0)
    ).sum()

    incoherencias["points_accumulated > 0 pero total_flights = 0"] = (
        (df["total_flights"] == 0) & (df["points_accumulated"] > 0)
    ).sum()

    print("REVISIÓN DE INCOHERENCIAS LÓGICAS")
    for k, v in incoherencias.items():
        print(f"{k}: {v}")
        
    return incoherencias

In [43]:
revisar_incoherencias_vuelos(df);

REVISIÓN DE INCOHERENCIAS LÓGICAS
booked > total: 0
companions > total: 0
distance > 0 pero total_flights = 0: 0
points_accumulated > 0 pero total_flights = 0: 0


In [46]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 403760 entries, 0 to 405623
Data columns (total 26 columns):
 #   Column                       Non-Null Count   Dtype  
---  ------                       --------------   -----  
 0   loyalty_number               403760 non-null  int64  
 1   flight_year                  403760 non-null  int64  
 2   flight_month                 403760 non-null  int64  
 3   flights_booked               403760 non-null  int64  
 4   flights_with_companions      403760 non-null  int64  
 5   total_flights                403760 non-null  int64  
 6   distance                     403760 non-null  float64
 7   points_accumulated           403760 non-null  int64  
 8   points_redeemed              403760 non-null  int64  
 9   dollar_cost_points_redeemed  403760 non-null  float64
 10  country                      403760 non-null  object 
 11  province                     403760 non-null  object 
 12  city                         403760 non-null  object 
 13  post

#### **9. EXPORTACIÓN DEL DATASET LIMPIO**

- Se guarda el csv limpio y procesado en `data/processed/` para su reutilización en análisis posteriores.


In [45]:
output_path = "../data/processed/customer_loyalty_clean.csv"
df.to_csv(output_path, index=False)

print(f"Dataset limpio exportado a: {output_path}")

Dataset limpio exportado a: ../data/processed/customer_loyalty_clean.csv
